In [1]:
import params
from train import *

In [2]:
config = default_config

In [3]:
config.batch_size=8
config.img_size=240
config.log_preds=False
config.lr=0.0007973010330496756

In [4]:
set_seed(config.seed)
run = wandb.init(project="course_debug", entity=None, job_type="training", config=config)

# good practice to inject params using sweeps
config = wandb.config

# prepare data
processed_dataset_dir = download_data()
proc_df = get_df(processed_dataset_dir)
dls = get_data(proc_df, bs=config.batch_size, img_size=config.img_size, augment=config.augment)

metrics = [MIOU(), BackgroundIOU(), RoadIOU(), TrafficLightIOU(),
           TrafficSignIOU(), PersonIOU(), VehicleIOU(), BicycleIOU()]

cbs = [WandbCallback(log_preds=False, log_model=True), 
       SaveModelCallback(monitor='miou'),] + ([MixedPrecision()] if config.mixed_precision else [])

learn = unet_learner(dls, arch=getattr(tvmodels, config.arch), pretrained=config.pretrained, 
                     metrics=metrics)

learn.fit_one_cycle(config.epochs, config.lr, cbs=cbs)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: capecape. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact bdd_simple_1k_split:latest, 813.24MB. 4010 files... 
wandb:   4010 of 4010 files downloaded.  
Done. 0:0:0.6


epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.515155,0.418262,0.296839,0.818853,0.662946,0.000000,0.000000,0.000000,0.596073,0.000000,00:19
1,0.386973,0.343248,0.335863,0.888760,0.787141,0.000000,0.000000,0.000000,0.675138,0.000000,00:15
2,0.314943,0.268625,0.344038,0.898275,0.800011,0.000000,0.000000,0.000000,0.709979,0.000000,00:14
3,0.277079,0.244932,0.350016,0.903860,0.806253,0.000000,0.000000,0.000000,0.739999,0.000000,00:14
4,0.271980,0.251401,0.358115,0.913878,0.834959,0.020116,0.000000,0.000000,0.737854,0.000000,00:14
5,0.237291,0.233968,0.373437,0.916962,0.824694,0.124572,0.000000,0.000000,0.747830,0.000000,00:14
6,0.209146,0.203995,0.384149,0.925775,0.860155,0.132936,0.000000,0.000000,0.770178,0.000000,00:15
7,0.183490,0.203358,0.380567,0.929143,0.865715,0.087085,0.000151,0.000000,0.781873,0.000000,00:14
8,0.173932,0.202937,0.386216,0.929320,0.865572,0.123021,0.000604,0.000000,0.784993,0.000000,00:15
9,0.162763,0.201506,0.391280,0.929316,0.866574,0.157592,0.001356,0.000000,0.784120,0.000000,00:14


Better model found at epoch 0 with miou value: 0.2968387334206581.
Better model found at epoch 1 with miou value: 0.3358627074584826.
Better model found at epoch 2 with miou value: 0.34403774217582866.
Better model found at epoch 3 with miou value: 0.35001605669994973.
Better model found at epoch 4 with miou value: 0.35811536459532656.
Better model found at epoch 5 with miou value: 0.3734368071193299.
Better model found at epoch 6 with miou value: 0.38414912063548395.
Better model found at epoch 8 with miou value: 0.38621555895827936.
Better model found at epoch 9 with miou value: 0.3912798539746145.


In [5]:
scores = learn.validate()

In [6]:
scores

(#9) [0.20155881345272064,0.39130715261627863,0.9292930238040242,0.866520402172688,0.15786954738137562,0.0013940170296134428,0.0,0.7840730779262495,0.0]

In [9]:
m = [m.name for m in learn.metrics]
m

['miou',
 'background_iou',
 'road_iou',
 'traffic_light_iou',
 'traffic_sign_iou',
 'person_iou',
 'vehicle_iou',
 'bicycle_iou']

In [ ]:
final_metrics(learn)